In [2]:
pauli_ops = {}
pauli_ops['X'], pauli_ops['Y'], pauli_ops['Z'], pauli_ops['I'] = 1,2,3,4

In [3]:
a = ['I', 'am', 'dumb']

for index, string in enumerate(a):
    print(index, string)

0 I
1 am
2 dumb


In [4]:
print(pauli_ops['I'])

4


In [ ]:
# in time_evolution
def apply_rotations(state0, params):
    """Apply local single-qubit rotations defined in `params`.

    Expected `params` to contain a key "rot_angles" with either shape (L,3)
    giving per-qubit rotation angles (ax, ay, az) in radians, or shape (3,) to
    apply the same rotation to all qubits. The rotations are applied as
    Rx(ax) @ Ry(ay) @ Rz(az) per qubit and the full unitary returned.

    Returns the full unitary U (2**L x 2**L) so callers can do H -> U H U^\n and
    operator transforms O -> U O U^\n easily.
    """
    # Support both passing params dict or passing rot_angles array directly
    if isinstance(params, dict):
        rot_angles = params.get("rot_angles", None)
    else:
        rot_angles = params

    if rot_angles is None:
        return None

    rot_angles = jnp.array(rot_angles)

    # Determine L from angles
    if rot_angles.ndim == 1 and rot_angles.size == 3:
        # same rotation for all qubits; caller must provide L separately later
        raise ValueError("When providing a single (3,) rotation vector, call apply_rotations with per-qubit angles or use apply_rotations_unitary(L, rot_angles) to expand.")

    L = rot_angles.shape[0]

    # Single-qubit rotation matrices
    def Rx(a):
        return jnp.array([[jnp.cos(a/2), -1j*jnp.sin(a/2)], [-1j*jnp.sin(a/2), jnp.cos(a/2)]], dtype=jnp.complex64)
    def Ry(a):
        return jnp.array([[jnp.cos(a/2), -jnp.sin(a/2)], [jnp.sin(a/2), jnp.cos(a/2)]], dtype=jnp.complex64)
    def Rz(a):
        return jnp.array([[jnp.exp(-0.5j*a), 0.0], [0.0, jnp.exp(0.5j*a)]], dtype=jnp.complex64)

    # Build per-qubit unitaries and tensor-product them
    U_list = []
    for i in range(L):
        ax, ay, az = rot_angles[i]
        # Order: Rx(ax) then Ry(ay) then Rz(az) (applied right-to-left)
        U1 = Rx(ax) @ Ry(ay) @ Rz(az)
        U_list.append(U1)

    # Tensor product
    def kron_n(ops):
        out = ops[0]
        for A in ops[1:]:
            out = jnp.kron(out, A)
        return out

    U_full = kron_n(U_list)
    return U_full

In [ ]:
#in make_rhs_schrodinger          
            H0 = xyz_hamiltonian_from_theta(L, params["theta"], OPS_XYZ, hamiltonian_type)
            # If rotation angles are provided, apply single-qubit rotations before evolution
            if params is not None and isinstance(params, dict) and ("rot_angles" in params):
                U = apply_rotations(params)
                H0 = U @ H0 @ U.conj().T
            return H0



#in create mask
    # Rotations (optional) - controlled by train_theta flag by default; you can
    # change this to a separate config flag if desired
    if "rot_angles" in params:
        mask["rot_angles"] = jnp.ones_like(params["rot_angles"], dtype=bool) if train_theta else jnp.zeros_like(params["rot_angles"], dtype=bool)
    return mask


#in main script after defining params variable

    # Initialize rotation angles (optional)
    if CONFIG.get("use_rotations", False):
        # rot_angles shape: (L, 3) -> (ax, ay, az) per qubit
        rot_init = jnp.zeros((L, 3), dtype=jnp.float32)
        params["rot_angles"] = rot_init
        print(f"  Initialized rot_angles with shape {rot_init.shape}")

    if use_noisy and CONFIG["learn_noise_params"]: